### GNN embedding

In [1]:
#!pip install python-chess


In [2]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
#!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cpu.html
#!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.6.0+cpu.html
#!pip install torch-geometric

In [3]:
import os
import zipfile
import chess.pgn
import torch
import numpy as np
from io import TextIOWrapper
from torch_geometric.data import Data
import pandas as pd



# =====================
# FUNCIÓN DE FEATURES
# =====================

def board_to_feature(board):
    """Convierte el estado del tablero a un vector de 773 features"""
    piece_map = board.piece_map()
    planes = np.zeros((12, 64), dtype=np.float32)
    piece_to_index = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }
    for square, piece in piece_map.items():
        planes[piece_to_index[piece.symbol()]][square] = 1.0
    flat_pieces = planes.reshape(-1)  # 768

    extras = [
        float(board.turn),
        float(board.has_kingside_castling_rights(chess.WHITE)),
        float(board.has_queenside_castling_rights(chess.WHITE)),
        float(board.has_kingside_castling_rights(chess.BLACK)),
        float(board.has_queenside_castling_rights(chess.BLACK)),
    ]
    return np.concatenate([flat_pieces, extras])  # 773

# =====================
# FUNCIÓN PARA GRAFO DE PARTIDA
# =====================

def pgn_to_graph_one_player(game, color="white"):
    board = game.board()
    x = []
    edge_index = [[], []]
    node_idx = 0
    move_idx = 0

    for move in game.mainline_moves():
        board.push(move)
        if (color == "white" and move_idx % 2 == 0) or (color == "black" and move_idx % 2 == 1):
            x.append(board_to_feature(board.copy()))
            if node_idx > 0:
                edge_index[0].append(node_idx - 1)
                edge_index[1].append(node_idx)
            node_idx += 1
        move_idx += 1

    if len(x) < 2:
        return None

    x = torch.tensor(np.stack(x), dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    return Data(x=x, edge_index=edge_index)

def process_zip_pgns(zip_folder):
    data_by_player = {}
    metadata = []
    game_id = 0

    for filename in os.listdir(zip_folder):
        if not filename.endswith(".zip"):
            continue

        player_name = filename.replace(".zip", "")
        zip_path = os.path.join(zip_folder, filename)
        graphs = []

        with zipfile.ZipFile(zip_path, 'r') as zf:
            for pgn_filename in zf.namelist():
                with zf.open(pgn_filename) as file:
                    pgn = TextIOWrapper(file, encoding='utf-8', errors='ignore')

                    while True:
                        game = chess.pgn.read_game(pgn)
                        if game is None:
                            break

                        white = game.headers.get("White", "").lower()
                        black = game.headers.get("Black", "").lower()
                        target = player_name.lower()

                        if target in white:
                            color = "white"
                        elif target in black:
                            color = "black"
                        else:
                            continue

                        graph = pgn_to_graph_one_player(game, color=color)
                        if graph:
                            graph.player = player_name
                            graph.game_id = game_id
                            graphs.append(graph)

                            metadata.append({
    "game_id": game_id,
    "player": player_name,
    "Event": game.headers.get("Event", ""),
    "Site": game.headers.get("Site", ""),
    "Date": game.headers.get("Date", ""),
    "Round": game.headers.get("Round", ""),
    "White": game.headers.get("White", ""),
    "Black": game.headers.get("Black", ""),
    "Result": game.headers.get("Result", ""),
    "WhiteElo": game.headers.get("WhiteElo", ""),
    "BlackElo": game.headers.get("BlackElo", ""),
    "ECO": game.headers.get("ECO", ""),
    "pgn": str(game)
})

                            game_id += 1

        data_by_player[player_name] = graphs
        print(f"{player_name}: {len(graphs)} grafos generados")

    # Guardar metadata
    df_meta = pd.DataFrame(metadata)
    df_meta.to_csv("games_metadata.csv", index=False)
    print("✅ Metadata guardada en games_metadata.csv")

    return data_by_player



In [5]:
graphs = process_zip_pgns("pgns")

Alekhine: 1660 grafos generados
Anand: 4201 grafos generados
Anderssen: 680 grafos generados
Aronian: 5106 grafos generados
Bogoljubow: 972 grafos generados
Botvinnik: 891 grafos generados
Bronstein: 1928 grafos generados
Capablanca: 597 grafos generados
Carlsen: 6613 grafos generados
Caruana: 5339 grafos generados
Chigorin: 687 grafos generados
DeLaBourdonnais: 0 grafos generados
Euwe: 1121 grafos generados
Fine: 304 grafos generados
Fischer: 825 grafos generados


illegal san: 'Qxe1' in r2k3r/2pPp3/p4n2/3b2B1/1p5P/2qP4/3RQ1P1/4K2R w - - 2 31 while parsing <Game at 0x24b1f666850 ('Gelfand,B' vs. 'Gareev,T', '2019.12.29' at 'Moscow RUS')>


Gelfand: 3969 grafos generados
Geller: 2195 grafos generados
Ivanchuk: 4949 grafos generados
Kamsky: 7035 grafos generados
Karjakin: 3534 grafos generados
Karpov: 3528 grafos generados
Kasparov: 2127 grafos generados
Keres: 1570 grafos generados
Korchnoi: 1038 grafos generados
Kramnik: 4323 grafos generados
Larsen: 2377 grafos generados
Lasker: 899 grafos generados
Leko: 2679 grafos generados
Maroczy: 754 grafos generados
Morphy: 211 grafos generados
Najdorf: 1603 grafos generados
Nimzowitsch: 511 grafos generados
Petrosian: 1892 grafos generados
Philidor: 6 grafos generados
Pillsbury: 387 grafos generados
Polugaevsky: 1889 grafos generados
Portisch: 3029 grafos generados
Reshevsky: 1265 grafos generados
Rubinstein: 796 grafos generados
Schlechter: 738 grafos generados
Smyslov: 2624 grafos generados
Spassky: 2229 grafos generados
Staunton: 283 grafos generados
Steinitz: 589 grafos generados
Tal: 2430 grafos generados
Tarrasch: 703 grafos generados
Timman: 3618 grafos generados
Topalov:

In [11]:
graphs_list = []
game_id_counter = 0

for player, data_list in graphs.items():
    for data in data_list:
        data.player = player
        data.game_id = game_id_counter
        graphs_list.append(data)
        game_id_counter += 1
    

In [12]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
import pandas as pd
from tqdm import tqdm

# ======================
# Hiperparámetros
# ======================
INPUT_DIM = 773
HIDDEN_DIM = 256
EMBED_DIM = 128
BATCH_SIZE = 32

# ======================
# Modelo GCN Encoder
# ======================
class GNNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return global_mean_pool(x, batch)

# ======================
# Cargar grafos
# ======================
#graphs = torch.load("chess_graphs.pt")  # cada Data debe tener .player y .game_id
loader = DataLoader(graphs_list, batch_size=BATCH_SIZE, shuffle=False)

model = GNNEncoder(INPUT_DIM, HIDDEN_DIM, EMBED_DIM)
model.eval()

# ======================
# Embedding loop
# ======================
embeddings = []
players = []
game_ids = []

with torch.no_grad():
    for batch in tqdm(loader, desc="Generando embeddings"):
        z = model(batch.x, batch.edge_index, batch.batch)
        embeddings.append(z)
        graphs_in_batch = batch.to_data_list()
        players += [g.player for g in graphs_in_batch]
        game_ids += [g.game_id for g in graphs_in_batch]

embeddings = torch.cat(embeddings).cpu().numpy()

# ======================
# Guardar resultados
# ======================
df = pd.DataFrame(embeddings, columns=[f"dim_{i}" for i in range(EMBED_DIM)])
df["player"] = players
df["game_id"] = game_ids
df.to_csv("gnn_chess_embeddings.csv", index=False)

print("✅ Embeddings GNN guardados como CSV.")



Generando embeddings:   0%|          | 0/3120 [00:00<?, ?it/s]

Generando embeddings: 100%|██████████| 3120/3120 [01:28<00:00, 35.06it/s]


✅ Embeddings GNN guardados como CSV.
